In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from transformers import AutoTokenizer, AutoModel
import torch
import urllib.parse

d:\workspace\Chunjae_Bigdata_9th\9.4~9.23\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [60]:
# 드라이버 설정
start_url = r'https://ko.wikipedia.org/wiki/파이썬'
target_url = r'https://ko.wikipedia.org/wiki/안젤리나_졸리'

# 목표 URL에서 마지막 부분 추출 및 띄어쓰기 처리
target_label = urllib.parse.unquote(target_url.split('/')[-1]).replace('_', ' ')

tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
model = AutoModel.from_pretrained("beomi/kcbert-base")

driver = webdriver.Chrome()
driver.get(start_url)

def get_text_features(text):
    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
    with torch.no_grad():
        features = model(**encoded_input)
    return features.last_hidden_state.squeeze().mean(dim=0)

target_features = get_text_features(target_label)  # 목표 링크 텍스트 특성

# 방문한 페이지를 추적하기 위한 집합
visited_links = set()

# 문서 존재 여부 확인 (a 태그의 title 속성에 "(없는 문서)" 확인)
def is_valid_link(link):
    link_title = link.get_attribute('title')
    if link_title and "(없는 문서)" in link_title:
        return False  # 문서가 존재하지 않음
    return True  # 문서가 존재

# 페이지 본문 길이 확인 함수
def is_valid_length(content_div, min_length=500):
    page_text = content_div.text
    return len(page_text) > min_length

# 탐색 함수
def navigate_to_target(driver, max_depth=50, min_length=100):
    depth = 0
    while depth < max_depth:  # 탐색 깊이를 제한하여 더 많은 시도를 하도록 설정
        current_url = driver.current_url
        decoded_current_url = urllib.parse.unquote(current_url).replace('_', ' ')
        decoded_target_url = urllib.parse.unquote(target_url).replace('_', ' ')

        if decoded_current_url == decoded_target_url:
            print("목표 페이지에 도달했습니다. 프로세스를 종료합니다.")
            print(depth)
            return True  # 목표에 도달하면 탐색 종료

        visited_links.add(current_url)  # 현재 URL을 방문 목록에 추가

        # mw-content-ltr mw-parser-output 클래스를 가진 div 태그 내의 링크만 선택
        content_div = driver.find_element(By.ID, "bodyContent")
        links = content_div.find_elements(By.TAG_NAME, 'a')
        
        link_scores = {}

        # 각 링크를 평가
        for link in links:
            link_text = link.text
            link_url = link.get_attribute('href')

            # 링크 필터링: 위키미디어 공용, 각주 (#cite 포함), "편집" 텍스트, 한국어 위키 외 링크, 이미 방문한 링크, 없는 문서 제외
            if (link_text and link_url 
                and 'ko.wikipedia.org' in link_url  # 한국어 위키 문서만 허용
                and 'wiki' in link_url  # 위키 링크만 허용
                and '위키미디어 공용' not in link_text  # "위키미디어 공용" 링크 제외
                and '#cite' not in link_url  # 각주 링크 제외
                and link_text != "편집"  # "편집" 텍스트 제외
                and not link_url.startswith(r"https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:")  # "위키백과:"로 시작하는 URL 제외
                and link_url not in visited_links  # 이미 방문한 링크 제외
                and is_valid_link(link)):  # 없는 문서 필터링
                
                # 코사인 유사도 계산
                link_features = get_text_features(link_text)
                cos_sim = torch.nn.functional.cosine_similarity(target_features, link_features, dim=0)
                link_scores[link_url] = cos_sim.item()

        if not link_scores:
            print("더 이상 탐색할 링크가 없습니다.")
            depth += 1
            continue  # 유사도가 낮더라도 계속 탐색

        # 가장 높은 점수의 링크 선택 (최소 유사도 기준을 적용하지 않음)
        best_link = max(link_scores, key=link_scores.get)
        highest_score = link_scores[best_link]

        # 본문 길이 체크 (본문이 너무 짧은 경우 뒤로 가기)
        if best_link not in visited_links:
            driver.get(best_link)
            visited_links.add(best_link)  # 선택한 링크를 방문한 목록에 추가

            content_div = driver.find_element(By.ID, "bodyContent")

            if not is_valid_length(content_div, min_length):
                print(f"본문 길이가 {min_length}자 이하인 문서는 무시하고 뒤로 돌아갑니다: {best_link}")
                driver.back()  # 뒤로 가기
                visited_links.add(best_link)  # 무시한 링크를 방문한 목록에 추가
                depth += 1
                continue  # 다른 링크로 이동
            else:
                print(f"이동할 링크: {best_link} - 유사도: {highest_score:.4f}")
        else:
            print(f"이미 방문한 링크를 무시합니다: {best_link}")

        depth += 1

    print("탐색 깊이 한도에 도달하여 프로세스를 종료합니다.")
    return False

navigate_to_target(driver)


이동할 링크: https://ko.wikipedia.org/wiki/%EA%B7%B8%EB%A3%A8%EB%B9%84_(%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D_%EC%96%B8%EC%96%B4) - 유사도: 0.5434
이동할 링크: https://ko.wikipedia.org/wiki/%EA%B7%B8%EB%A0%88%EC%9D%BC%EC%A6%88 - 유사도: 0.5457
이동할 링크: https://ko.wikipedia.org/wiki/%EB%9D%BC%EB%AF%B8%EB%82%98%EC%8A%A4 - 유사도: 0.5754
이동할 링크: https://ko.wikipedia.org/wiki/%EB%9D%BC%EC%9A%B0%ED%84%B0 - 유사도: 0.5628
이동할 링크: https://ko.wikipedia.org/wiki/%EB%AC%B4%EC%84%A0_%EB%9D%BC%EC%9A%B0%ED%84%B0 - 유사도: 0.5879
이동할 링크: https://ko.wikipedia.org/wiki/%EC%9B%A8%EC%9D%B4%EB%B0%B1_%EB%A8%B8%EC%8B%A0 - 유사도: 0.5182
이동할 링크: https://ko.wikipedia.org/wiki/%EA%B0%A4%EB%9F%AC%EB%A6%AC - 유사도: 0.5410
이동할 링크: https://ko.wikipedia.org/wiki/%EC%BD%94%EC%8B%9C%EB%AA%A8_%EB%8D%B0_%EB%A9%94%EB%94%94%EC%B9%98 - 유사도: 0.5265
이동할 링크: https://ko.wikipedia.org/wiki/%ED%94%BC%EC%B9%B4%EB%A5%B4%EB%8B%A4_%EB%B6%80%EC%97%90%EB%A6%AC - 유사도: 0.5899
이동할 링크: https://ko.wikipedia.org/wiki/%EB%A1%9C%EB%A0%8C%EC%B4%88_%EC%9D%BC_%EB%B2

True